

> Import libraries to use



In [2]:
import numpy as np

>  # Introduction to numpy (Skip if you already are familiar)

>> Creating a 1D array

In [3]:
a = np.array([1,2,3,4])
print(a)

[1 2 3 4]


>> Creating a 2D array


In [4]:
a = np.array([[1,2],[3,4]])
print(a)

[[1 2]
 [3 4]]


>> Creating an array full of zeros


In [5]:
a = np.zeros(shape=(10))
print(a)
a = np.zeros(shape=(5,2))
print(a)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


>> Infinity in numpy

In [6]:
print(np.inf)

inf


>> Max and Argmax

In [7]:
a = np.array([2,1,4,3])
print(np.max(a))
print(np.argmax(a))

4
2


>> From list to Numpy

In [8]:
l = [1,2,3,4]
print(l)
print(np.asarray(l))

[1, 2, 3, 4]
[1 2 3 4]


>> Random in numpy

In [9]:
# Array of Random integers ranging from 1 to 10 (with any size you want)
a = np.random.randint(low=1, high=10, size=(5,2))
print(a)

# Array of random elements of a list with any size you want
a = np.random.choice([0,1,2], size=(2,))

[[3 3]
 [3 3]
 [1 1]
 [8 8]
 [6 6]]


>> Shapes in numpy

In [10]:
a = np.random.randint(low=1, high=5, size=(4,2))
print(a.shape)
print(a)

# Reshape a to a vector of shape = (8,1)
a = a.reshape((8,1))
print(a.shape)
print(a)

(4, 2)
[[2 2]
 [4 1]
 [1 4]
 [3 3]]
(8, 1)
[[2]
 [2]
 [4]
 [1]
 [1]
 [4]
 [3]
 [3]]


# Pre-defined utilities

In [11]:

int_to_char = {
    0 : 'u',
    1 : 'r',
    2 : 'd',
    3 : 'l'
}

policy_one_step_look_ahead = {
    0 : [-1,0],
    1 : [0,1],
    2 : [1,0],
    3 : [0,-1]
}

def policy_int_to_char(pi,n):

    pi_char = ['']

    for i in range(n):
        for j in range(n):

            if i == 0 and j == 0 or i == n-1 and j == n-1:

                continue

            pi_char.append(int_to_char[pi[i,j]])

    pi_char.append('')

    return np.asarray(pi_char).reshape(n,n)

# 1- Policy evaluation

In [12]:
def policy_evaluation(n, pi, v, gamma, threshold):
    while True:
        delta = 0
        for i in range(n):
            for j in range(n):
                if (i == 0 and j == 0) or (i == n-1 and j == n-1):
                    continue  # Skip terminal states

                v_old = v[i, j]
                action = pi[i, j]
                ni, nj = i + policy_one_step_look_ahead[action][0], j + policy_one_step_look_ahead[action][1]

                # Check if new position is within bounds
                if 0 <= ni < n and 0 <= nj < n:
                    v[i, j] = -1 + gamma * v[ni, nj]
                else:
                    v[i, j] = -1 + gamma * v[i, j]  # Stay in the current position

                delta = max(delta, abs(v_old - v[i, j]))

        if delta < threshold:
            break

    return v


# 2- Policy improvement

In [13]:
def policy_improvement(n, pi, v, gamma):
    policy_stable = True
    new_pi = np.copy(pi)
    for i in range(n):
        for j in range(n):
            if (i == 0 and j == 0) or (i == n-1 and j == n-1):
                continue

            old_action = pi[i, j]
            max_value = float('-inf')

            for action in range(4):  # UP=0, RIGHT=1, DOWN=2, LEFT=3
                ni, nj = i + policy_one_step_look_ahead[action][0], j + policy_one_step_look_ahead[action][1]

                if 0 <= ni < n and 0 <= nj < n:
                    value = -1 + gamma * v[ni, nj]
                else:
                    value = -1 + gamma * v[i, j]

                if value > max_value:
                    max_value = value
                    new_pi[i, j] = action

            if old_action != new_pi[i, j]:
                policy_stable = False

    return new_pi, policy_stable


# 3- Policy Initialization

In [14]:
def policy_initialization(n):
    # Randomly initialize the policy for each state (excluding terminal states)
    pi = np.random.randint(0, 4, size=(n, n))
    pi[0, 0] = pi[n-1, n-1] = 0  # Set terminal states to any fixed value (e.g., 0)
    return pi


# 4- Policy Iteration algorithm

In [15]:
def policy_iteration(n, Gamma, threshhold):
    pi = policy_initialization(n)
    v = np.zeros((n, n))

    while True:
        v = policy_evaluation(n, pi, v, Gamma, threshhold)  # Correct the parameter names
        pi, pi_stable = policy_improvement(n, pi, v, Gamma)

        if pi_stable:
            break

    return pi, v


# Main Code to Test

In [16]:
n = 4

Gamma = [0.8,0.9,1]

threshhold = 1e-4

for _gamma in Gamma:

    pi , v = policy_iteration(n=n,Gamma=_gamma,threshhold=threshhold)

    pi_char = policy_int_to_char(n=n,pi=pi)

    print()
    print("Gamma = ",_gamma)

    print()

    print(pi_char)

    print()
    print()

    print(v)



Gamma =  0.8

[['' 'l' 'l' 'd']
 ['u' 'u' 'u' 'd']
 ['u' 'u' 'r' 'd']
 ['u' 'r' 'r' '']]


[[ 0.   -1.   -1.8  -2.44]
 [-1.   -1.8  -2.44 -1.8 ]
 [-1.8  -2.44 -1.8  -1.  ]
 [-2.44 -1.8  -1.    0.  ]]

Gamma =  0.9

[['' 'l' 'l' 'd']
 ['u' 'u' 'u' 'd']
 ['u' 'u' 'r' 'd']
 ['u' 'r' 'r' '']]


[[ 0.   -1.   -1.9  -2.71]
 [-1.   -1.9  -2.71 -1.9 ]
 [-1.9  -2.71 -1.9  -1.  ]
 [-2.71 -1.9  -1.    0.  ]]


KeyboardInterrupt: 